In [ ]:
from transformers import AutoModel

# 加载模型
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModel.from_pretrained(model_name, cache_dir="/cache/huggingface/hub")

# 计算总参数量
total_params = sum(p.numel() for p in model.parameters())

print(f"模型的总参数量: {total_params}")

# 比较模型参数差异并提取位置

In [ ]:
from typing import Union, List
from pprint import pprint
import torch

from transformers import AutoModelForCausalLM


def compare_mlp_params(model1, model2, mlp_layers: Union[str, List[str]]) -> dict:
    """比较多个MLP层的参数差异

    Args:
        mlp_layers: 支持以下格式：
            - 单个层模式: "transformer.h.0.mlp"
            - 多个层模式: ["transformer.h.0.mlp", "transformer.h.5.mlp"]
    """
    # 统一处理为列表格式
    if isinstance(mlp_layers, str):
        target_patterns = [mlp_layers]
    else:
        target_patterns = mlp_layers

    # 多模式参数提取
    def filter_params(model):
        return {
            name: param
            for name, param in model.named_parameters()
            if any(pattern in name for pattern in target_patterns)
        }

    params1 = filter_params(model1)
    params2 = filter_params(model2)

    # 结构一致性检查
    if params1.keys() != params2.keys():
        missing_in_1 = set(params2.keys()) - set(params1.keys())
        missing_in_2 = set(params1.keys()) - set(params2.keys())
        raise ValueError(
            f"模型结构不一致\n"
            f"Model1缺失层: {list(missing_in_1)}\n"
            f"Model2缺失层: {list(missing_in_2)}"
        )

    differences = {}

    for name in params1:
        p1, p2 = params1[name].cpu(), params2[name].cpu()

        if p1.shape != p2.shape:
            raise ValueError(f"形状不匹配: {name} | {p1.shape} vs {p2.shape}")

        if not torch.equal(p1, p2):
            diff_mask = ~torch.isclose(p1, p2, rtol=1e-5, atol=1e-8)
            diff_indices = torch.unique(torch.nonzero(diff_mask)[:, 0])

            differences[name] = {
                "shape": tuple(p1.shape),
                "diff_ratio": diff_indices.size(0) / p1.numel(),
                "diff_indices": diff_indices.tolist(),
            }

    return differences


# 使用示例
if __name__ == "__main__":
    # 单层比较
    # diff = compare_mlp_params(model1_path, model2_path, "transformer.h.0.mlp")
    model1_path = "Qwen/Qwen2.5-0.5B-Instruct"
    model2_path = "/cache/models/target_neurons_10R_reranker/results/target_neurons/checkpoint-285430"
    model1 = AutoModelForCausalLM.from_pretrained(
        model1_path, torch_dtype=torch.bfloat16
    )
    model2 = AutoModelForCausalLM.from_pretrained(
        model2_path, torch_dtype=torch.bfloat16
    )
    # for name, param in model1.named_parameters():
    #     print(name)
    target_modules = []
    for idx, layer in enumerate(model1.model.layers):
        module_str = f"model.layers.{idx}.mlp.down_proj.weight"
        target_modules.append(module_str)
    # 多层比较
    diff = compare_mlp_params(model1, model2, mlp_layers=target_modules)
    pprint(diff)